In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sbe_vallib import BinaryValidation, BinaryScorer, BinarySampler

In [3]:
from sklearn.metrics import recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


def recall_at_10(true, preds):
    return recall_score(true, preds)

def custom_test(sampler, scorer):
    pass
    

X, y = make_classification(n_samples=1000, n_classes=2,
                               n_features=5, n_informative=3, n_redundant=0,
                               random_state=0)

# X = pd.DataFrame(X)
# y = pd.Series(y)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

model = LogisticRegression().fit(X_train, y_train)


sampler = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=False)

sampler.source_state = False

In [5]:
sampler.set_seed(17)
sampler.oos['X'][:5], sampler.oos['y_true'][:5], sampler.oos['y_pred'][:5], sampler.train['X'][:5], sampler.train['y_true'][:5], sampler.train['y_pred'][:5]

(array([[-2.17494019, -1.94882167,  1.51645859, -0.07984927,  0.36288949],
        [-0.29589373, -0.60658754,  1.3077079 , -0.96369018, -0.36141957],
        [ 0.90220379, -0.73748739,  2.22030551,  0.13522097, -1.15197079],
        [ 0.85614706, -1.46948688, -0.89728366, -0.67901244, -0.25092918],
        [ 1.62796243,  0.76685632,  0.76080776,  0.6368111 ,  0.27275829]]),
 array([1, 1, 1, 0, 1]),
 array([[0.20805274, 0.79194726],
        [0.2338746 , 0.7661254 ],
        [0.05374312, 0.94625688],
        [0.78686938, 0.21313062],
        [0.31976163, 0.68023837]]),
 array([[ 0.91198054,  0.50023103,  1.99986692, -0.99729165,  0.15365226],
        [-1.39793602,  1.06997912,  0.71315234, -0.65700197, -2.46928164],
        [-0.9710823 , -1.05232521,  1.19846279, -1.13319911,  0.07803667],
        [ 0.12770901,  2.96796354,  0.62891825,  0.57968663, -0.12095324],
        [-0.5096907 , -0.09935072, -1.06533583,  0.98910109, -0.13231792]]),
 array([1, 0, 1, 0, 0]),
 array([[0.0906229 , 0.9

In [6]:
sampler = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=True)
sampler.source_state = False
sampler.set_seed(10)
sampler.oos['X'][:5], sampler.oos['y_true'][:5], sampler.oos['y_pred'][:5], sampler.train['X'][:5], sampler.train['y_true'][:5], sampler.train['y_pred'][:5]

(array([[-2.40496885, -1.81918034,  0.83077319,  0.14462519, -0.89298611],
        [ 0.95901739,  0.75327655,  1.73804534,  1.02667522,  0.54307603],
        [-2.31069337, -0.65141381, -0.55146407, -0.08948511,  0.48757919],
        [ 0.86694933,  0.7423848 ,  0.50661052, -1.22479637, -0.80471875],
        [ 0.62240996, -0.41502841,  1.7575855 ,  0.26581643, -0.84589129]]),
 array([1, 1, 1, 1, 1]),
 array([[0.46771723, 0.53228277],
        [0.12303249, 0.87696751],
        [0.88178655, 0.11821345],
        [0.50077952, 0.49922048],
        [0.11320953, 0.88679047]]),
 array([[-2.11797381, -2.65643153,  1.98797812, -0.83611109,  1.9991244 ],
        [ 1.22302727,  3.28963287, -0.19194977,  1.0505019 , -0.28810814],
        [-1.26218091, -1.69659881,  1.48195458, -0.16440693,  1.13496002],
        [-0.10637598, -1.53370503, -1.06392014,  0.71386447, -0.98088364],
        [-0.90313707, -0.31791572,  0.41308821,  0.30952743,  0.37529998]]),
 array([1, 0, 1, 0, 1]),
 array([[0.08537598, 0.9

In [7]:
sampler = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=False)
sampler.oos['X'][:5], sampler.oos['y_true'][:5], sampler.oos['y_pred'][:5], sampler.train['X'][:5], sampler.train['y_true'][:5], sampler.train['y_pred'][:5]

(array([[-1.02926877, -1.37725543,  1.13479799,  0.25541285,  0.53175481],
        [ 0.41405846, -3.04211036, -1.74482642, -0.31495099,  0.04021318],
        [ 1.1881111 ,  1.61352461,  0.18361962,  0.83598963, -1.39500325],
        [ 1.30772546,  0.45192686,  1.0399803 ,  0.1326134 , -2.16841434],
        [-0.81768414,  0.12410398,  0.89148861, -1.11590759,  1.18990366]]),
 array([1, 0, 1, 1, 0]),
 array([[0.2621659 , 0.7378341 ],
        [0.8977842 , 0.1022158 ],
        [0.64869929, 0.35130071],
        [0.2924985 , 0.7075015 ],
        [0.41046284, 0.58953716]]),
 array([[-2.11797381, -2.65643153,  1.98797812, -0.83611109,  1.9991244 ],
        [ 1.22302727,  3.28963287, -0.19194977,  1.0505019 , -0.28810814],
        [-1.26218091, -1.69659881,  1.48195458, -0.16440693,  1.13496002],
        [-0.10637598, -1.53370503, -1.06392014,  0.71386447, -0.98088364],
        [-0.90313707, -0.31791572,  0.41308821,  0.30952743,  0.37529998]]),
 array([1, 0, 1, 0, 1]),
 array([[0.08537598, 0.9

In [9]:
sampler_boot = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=True)
sampler_no_boot = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                        oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=False)

[np.array_equal(sampler_boot.oos['X'][:5], sampler_no_boot.oos['X'][:5]),
 np.array_equal(sampler_boot.oos['y_true'][:5], sampler_no_boot.oos['y_true'][:5]), 
 np.array_equal(sampler_boot.oos['y_pred'][:5],sampler_boot.oos['y_pred'][:5]),
 np.array_equal(sampler_boot.train['X'][:5],sampler_boot.train['X'][:5]),
 np.array_equal(sampler_boot.train['y_true'][:5],sampler_boot.train['y_true'][:5]),
 np.array_equal(sampler_boot.train['y_pred'][:5], sampler_boot.train['y_pred'][:5])]

[True, True, True, True, True, True]

In [181]:
from scipy.special import rel_entr

In [183]:
rel_entr(np.array([1, 2, 3]), np.array([1, 2, 1e-5]))

array([ 0.        ,  0.        , 37.83461326])

In [184]:
from sbe_vallib.validation.table.general_tests.data_quality.test_psi_factor import test_factor_psi

In [197]:
sampler_boot = BinarySampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                             oos = {'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)}, bootstrap=True)
results = test_factor_psi(sampler_boot, merge_upto_quantile=0.075)

[53 53 54 53 53 54 53 53 53 53 53 53 54  8]
[53 53 53 53 53 53 53 53 53 53 53 53 53 11]
[53 53 53 53 53 53 53 53 53 53 53 53 53 11]
[54 53 53 53 53 53 53 53 53 53 53 53 53 10]
[53 53 53 53 53 53 53 53 53 53 53 53 53 11]


In [198]:
results['result_dataframes']

,0,1,2,3,4
psi,0.053377,0.157847,0.076086,0.098411,0.114026
feat_type,Continuous,Continuous,Continuous,Continuous,Continuous
bin_count,17,17,17,17,17
hist_train,"[0.000, 53.000, 53.000, 54.000, 53.000, 53.000...","[0.000, 53.000, 53.000, 53.000, 53.000, 53.000...","[0.000, 53.000, 53.000, 53.000, 53.000, 53.000...","[0.000, 54.000, 53.000, 53.000, 53.000, 53.000...","[0.000, 53.000, 53.000, 53.000, 53.000, 53.000..."
hist_oos,"[0.000, 26.000, 25.000, 27.000, 20.000, 17.000...","[2.000, 21.000, 18.000, 32.000, 10.000, 26.000...","[0.000, 23.000, 19.000, 22.000, 23.000, 34.000...","[0.000, 33.000, 28.000, 20.000, 22.000, 30.000...","[0.000, 34.000, 26.000, 25.000, 37.000, 11.000..."
bins,"[-inf, -4.06049, -2.00164, -1.49913, -1.04345,...","[-inf, -3.79651, -1.9582, -1.48804, -1.06203, ...","[-inf, -2.06933, -1.21815, -0.91757, -0.5612, ...","[-inf, -3.11696, -1.3541, -0.93572, -0.67664, ...","[-inf, -3.82495, -1.31814, -0.92793, -0.69203,..."
feature,0,1,2,3,4
semaphore,green,yellow,green,green,yellow


In [199]:
a, b = results['result_dataframes'][1]['hist_train'], results['result_dataframes'][1]['hist_oos']
a, b = np.array(a).astype(float), np.array(b).astype(float)


In [200]:
a / np.sum(a)

array([0.        , 0.07571429, 0.07571429, 0.07571429, 0.07571429,
       0.07571429, 0.07571429, 0.07571429, 0.07571429, 0.07571429,
       0.07571429, 0.07571429, 0.07571429, 0.07571429, 0.01571429,
       0.        ])

In [196]:
b / np.sum(b)

array([0.00666667, 0.07      , 0.04666667, 0.1       , 0.04666667,
       0.06666667, 0.07666667, 0.05      , 0.08      , 0.08666667,
       0.08333333, 0.03333333, 0.07      , 0.09666667, 0.08666667,
       0.        ])